In [46]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv('songdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [48]:
df.shape

(57650, 4)

In [49]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [50]:
df.shape

(5000, 3)

In [51]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [52]:
df['text'][0]

"seems every star is out tonight to light our love   so round the moon so full and bright so high above   but if you look real closely there's only me, there's only you   and when you hold me closely, yeah      and now the sky melts in the sea like they are one   it's hard to tell where one first starts and where ends one   that's the way our love is   you part of me me part of you      that's just what love is   sweet you got the sweetest emotion, baby   sweet, sweet emotion   sweet you got the sweetest emotion   sweeter than sweet   sweeter than sweet   sweeter than sweet      you took me from the surface to a deeper deep   showing me that you   and you are life for me   our love will last forever   me loving you, you loving me   we'll always be together oh      sweet you got the sweetest emotion   sweet, sweet emotion   sweet, sweet emotion   sweet you got the sweetest emotion   sweeter than sweet   sweeter than sweet   sweeter than sweet      sweet, you got the sweetest emotion   s

In [53]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [54]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [57]:
df['song'][0]

'Sweet Emotion'

In [58]:
similarity[0]

array([1.        , 0.01379092, 0.033895  , ..., 0.00543623, 0.01829951,
       0.0084287 ])

In [59]:
df[df['song']=='Sweet Emotion'].index[0]

0

# recommedation function

In [60]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [61]:
recommendation('Sweet Emotion')

['My Sugar So Sweet',
 'My Sweet Love',
 'Sweet Love',
 'Sweet Time',
 'So Emotional',
 'Let It Be Me',
 'Give Me A Kiss (Just One Sweet Kiss)',
 'How Sweet It Is',
 "Sweet Child O' Mine",
 'Just A Little Light',
 'For You Blue',
 'In The Sweet By And By',
 'Sweeter Than Fiction',
 'Sweetest Angel',
 "Don't Ease Me In",
 'Sweet Surrender',
 'Good Night',
 'Sweet Thing',
 'This Is Emotion',
 'Child Of Mine']

In [62]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [18]:
print(df['song'].unique())

['Have You Met Miss Jones?' 'Blue Velvet Band' 'Gone' ...
 'Christmas Auld Lang Syne' 'Room For The Life' 'Post Post-Modern Man']
